# Modern Data Structures Homework 3

Hsin-Ju (Lulu) Huang  
hh3105

In [33]:
import pandas as pd
import numpy as np
import scipy.stats as stats


flights = pd.read_csv(r"C:\Users\lulu\Documents\GitHub\QMSS-GR5072-Spring2025_Class\HW\HW03\nycflights.csv")

## Question 1
### [20 pts] Write a function that takes a single numerical variable and returns three values, the minimum number, the median, and the maximum number of the vector. Test your function using the month column of the flights data set.

In [34]:
# Define the function
def min_median_max(variable):
    return min(variable), float(np.median(variable)), max(variable)

# Apply function to the 'month' column
result = min_median_max(flights['month'])
print(result)  

(1, 7.0, 12)


## Question 2
### [5 pts] Explain your reasoning for choosing your function's name in the previous question.

I chose min_median_max as the function name to indicate that it returns the minimum, median, and maximum values of the input vector.

## Question 3
### [20 pts] Write a function that categorizes a numerical variable in the flights data into four categories.

In [35]:
# Define the function
def categorize_time_of_day(data, column):
    def classify_time(value):
        if pd.isna(value):  # Handle missing values
            return "Unknown"
        elif 500 <= value < 1200:
            return "Morning"
        elif 1200 <= value < 1700:
            return "Afternoon"
        elif 1700 <= value < 2100:
            return "Evening"
        else:
            return "Night"

    data[f"{column}_category"] = data[column].apply(classify_time)
    return data

# Apply function to the dep_time column
flights = categorize_time_of_day(flights, "dep_time")

# Print frequency table
print(flights["dep_time_category"].value_counts())

dep_time_category
Morning      129539
Afternoon     98617
Evening       79793
Night         20572
Unknown        8255
Name: count, dtype: int64


## Question 4
### [5 pts] Explain your reasoning for choosing your function's name in the previous question.

I chose the name categorize_time_of_day because the function is designed to categorize a given time into one of four parts of the day. The name clearly reflects its purpose of organizing times into distinct segments of the day.

## Question 5
### [20 pts] Write a function that calculates the median of all numeric variables in the flights dataset.

In [36]:
def median_num_columns(data):
    # Select only numeric columns
    numeric_data = data.select_dtypes(include='number')
    
    # Calculate the median for each numeric column
    medians = numeric_data.median()
    
    return medians

# Testing the function on the flights dataset
medians = median_num_columns(flights)
print(medians)

Unnamed: 0        168388.5
year                2013.0
month                  7.0
day                   16.0
dep_time            1401.0
sched_dep_time      1359.0
dep_delay             -2.0
arr_time            1535.0
sched_arr_time      1556.0
arr_delay             -5.0
flight              1496.0
air_time             129.0
distance             872.0
hour                  13.0
minute                29.0
dtype: float64


## Question 6
### [5 pts] Explain your reasoning for choosing your function's name in the previous question.

I chose the function name 'median_num_columns' because the function calculates the medians of all the numeric columns in the dataset.

## Question 7
### [20 pts] Modify the function t_test() we wrote in class together this week so that this function can handle violations to the homogeneity of variance (HOV) assumption.

In [37]:
def t_test(num_var, bin_var):  
    
    # Calculate the two groups based on the binary variable
    group1 = num_var[bin_var == 0]
    group2 = num_var[bin_var == 1]
    
    # Calculate the sample sizes and sample variances of the two groups
    n1 = group1.shape[0]
    n2 = group2.shape[0]
    s2_1 = group1.var()
    s2_2 = group2.var()
    
    # Calculate the sample means
    mean1 = group1.mean()
    mean2 = group2.mean()
    
    # Calculate the mean difference between the two groups
    mean_diff = mean2 - mean1
    
    # Calculate the ratio of variances
    var_ratio = s2_1 / s2_2 if s2_2 != 0 else np.inf
    
    # Check if the HOV assumption is violated
    if 0.25 <= var_ratio <= 4.00:
        # Homogeneity of variance assumption is not violated, use standard t-test
        
        # Calculate the degrees of freedom (DF)
        DF = n1 + n2 - 2
        
        # Calculate the pooled standard deviation (sp)
        sp = np.sqrt(((n1 - 1) * s2_1 + (n2 - 1) * s2_2) / DF)
        
        # Calculate the standard error of the mean difference
        SE_mean_diff = sp * np.sqrt(1/n1 + 1/n2)
        
        # Calculate the t-statistic
        t_statistic = mean_diff / SE_mean_diff
        
        # Calculate the p-value
        p_value = 2 * (1 - stats.t.cdf(np.abs(t_statistic), DF))
        
        # Store the results in a DataFrame
        results = pd.DataFrame({
            "Continuous Variable": [num_var.name],
            "Binary Variable": [bin_var.name],
            "Total Sample Size": [n1 + n2],
            "Mean Difference": [round(mean_diff, 2)], 
            "SE of Mean Difference": [round(SE_mean_diff, 2)],
            "DF": [DF],
            "t-statistic": [round(t_statistic, 3)],
            "P-value": [("%.3f" % p_value).lstrip('0')],
            "Test": "Independent samples t-test"
        })
        
        # Print the message for standard t-test
        print("The homogeneity of variance assumption was not violated, so an independent samples t-test was conducted.")
    
    else:
        # HOV assumption is violated, use Welch's t-test
        
        # Calculate the degrees of freedom (Welch-Satterthwaite equation)
        numerator = (s2_1 / n1 + s2_2 / n2) ** 2
        denominator = ((s2_1 / n1) ** 2) / (n1 - 1) + ((s2_2 / n2) ** 2) / (n2 - 1)
        DF_welch = numerator / denominator
        
        # Calculate Welch's t-statistic
        t_statistic_welch = mean_diff / np.sqrt(s2_1 / n1 + s2_2 / n2)
        
        # Calculate the p-value for Welch's t-test
        p_value_welch = 2 * (1 - stats.t.cdf(np.abs(t_statistic_welch), DF_welch))
        
        # Store the results in a DataFrame
        results = pd.DataFrame({
            "Continuous Variable": [num_var.name],
            "Binary Variable": [bin_var.name],
            "Total Sample Size": [n1 + n2],
            "Mean Difference": [round(mean_diff, 2)], 
            "SE of Mean Difference": [round(np.sqrt(s2_1 / n1 + s2_2 / n2), 2)],
            "DF": [round(DF_welch, 2)],
            "t-statistic": [round(t_statistic_welch, 3)],
            "P-value": [("%.3f" % p_value_welch).lstrip('0')],
            "Test": "Welch's t-test"
        })
        
        # Print the message for Welch's t-test
        print("The homogeneity of variance assumption was violated, so Welch's t-test was conducted.")
    
    return results

## Question 8
### [5 pts] Import the GED data set we used for the class activity. Call the t_test() function and test it out on the GED data set we used in class. Let the numeric variable be 'income_log' and let the binary variable be 'ged'.

In [39]:
# Import the GED dataset (assuming the file is in CSV format)
ged_data = pd.read_csv(r"C:\Users\lulu\Documents\GitHub\QMSS-GR5072-Spring2025_Class\HW\HW03\ged_data.csv")  # Replace with actual path

# Call the t_test function
results = t_test(ged_data['income_log'], ged_data['ged'])

# Print the results
print(results)

The homogeneity of variance assumption was not violated, so an independent samples t-test was conducted.
  Continuous Variable Binary Variable  Total Sample Size  Mean Difference  \
0          income_log             ged               5976            -0.48   

   SE of Mean Difference    DF  t-statistic P-value  \
0                   0.06  5974       -7.458    .000   

                         Test  
0  Independent samples t-test  
